# Import Required Libraries


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load Datasets

In [2]:
training_df = pd.read_csv("Training.csv")
symptom_severity_df = pd.read_csv("Symptom-severity.csv")
description_df = pd.read_csv("description.csv")
precautions_df = pd.read_csv("precautions_df.csv")
medications_df = pd.read_csv("medications.csv")
diets_df = pd.read_csv("diets.csv")
workout_df = pd.read_csv("workout_df.csv")

# Prepare training data

In [3]:
X = training_df.drop(columns=["prognosis"])
y = training_df["prognosis"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model

In [4]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

# Evaluate the Model

In [5]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model trained successfully with accuracy: {accuracy:.2f}")

Model trained successfully with accuracy: 1.00


# Utility functions

In [6]:
# List of available symptoms for validation
available_symptoms = X.columns.tolist()

def predict_disease(symptoms):
    """
    Predict the disease based on user-input symptoms.
    """
    # Convert symptoms to binary input for the model
    input_vector = np.zeros(len(X.columns))
    unrecognized_symptoms = []
    
    for symptom in symptoms:
        normalized_symptom = symptom.replace(" ", "_").strip().lower()
        if normalized_symptom in X.columns:
            input_vector[X.columns.get_loc(normalized_symptom)] = 1
        else:
            unrecognized_symptoms.append(symptom)

    if unrecognized_symptoms:
        print(f"\nWarning: The following symptoms were not recognized: {', '.join(unrecognized_symptoms)}")
        print(f"Available symptoms: {', '.join(available_symptoms)}")

    input_vector = input_vector.reshape(1, -1)
    predicted_disease = model.predict(input_vector)[0]
    confidence = np.max(model.predict_proba(input_vector))
    return predicted_disease, confidence

def provide_recommendations(disease):
    """
    Provide recommendations based on the predicted disease.
    """
    print(f"\nPredicted Disease: {disease}")
    
    # Description
    description = description_df.loc[description_df["Disease"] == disease, "Description"].values
    if description.size > 0:
        print(f"Description: {description[0]}")
    else:
        print("Description: No description available.")

    # Precautions
    precautions = precautions_df.loc[precautions_df["Disease"] == disease].values
    if precautions.size > 0:
        print("Precautions:")
        for i in range(1, 5):
            if precautions[0][i]:  # Skip empty precautions
                print(f"- {precautions[0][i]}")
    else:
        print("Precautions: No precautions available.")

    # Medications
    medications = medications_df.loc[medications_df["Disease"] == disease, "Medication"].values
    if medications.size > 0:
        print(f"Medications: {medications[0]}")
    else:
        print("Medications: No medications available.")

    # Diet
    diet = diets_df.loc[diets_df["Disease"] == disease, "Diet"].values
    if diet.size > 0:
        print(f"Diet: {diet[0]}")
    else:
        print("Diet: No diet information available.")

    # Workout
    workout = workout_df.loc[workout_df["disease"] == disease, "workout"].values
    if workout.size > 0:
        print(f"Workout: {workout[0]}")
    else:
        print("Workout: No workout information available.")

# Main function to interact with the user

In [ ]:
def main():
    """
    Main function to get user input and process it.
    """
    print("Welcome to the Disease Predictor!")
    input_symptoms = input("Enter symptoms separated by commas: ").strip().lower().split(",")
    input_symptoms = [symptom.strip() for symptom in input_symptoms]
    
    print(f"\nProcessing symptoms: {input_symptoms}")
    predicted_disease, confidence = predict_disease(input_symptoms)
    
    if confidence < 0.5:
        print(f"\nPrediction confidence is low ({confidence:.2f}). Results may not be accurate.")
    print(f"\nPredicted Disease: {predicted_disease} (Confidence: {confidence:.2f})")
    provide_recommendations(predicted_disease)

if __name__ == "__main__":
    main()

Welcome to the Disease Predictor!
